In [11]:
import s3fs
import polars as pl

In [12]:
fs = s3fs.S3FileSystem(anon=True)

In [38]:
files = fs.glob("noaa-gefs-pds/gefs.20240101/00/wave/*/*.idx")

df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        # .str.split("/")
        # .list.last()
        .str.replace(r"gefs.\d{8}/\d\d", "gefs.YYYYMMDD/HH")
        .str.replace(r"f\d\d\d", "f###")
        .str.replace(r"z.p\d\d", "z.p##")
        .str.replace(r".t\d\dz.", ".tHHz.")
        .alias("filename"),
    )
)
for i, f in enumerate(df["filename"].unique().sort(), start=1):
    print(f"{i:3}: {f}")

  1: noaa-gefs-pds/gefs.YYYYMMDD/HH/wave/gridded/gefs.wave.tHHz.c00.global.0p25.f###.grib2.idx
  2: noaa-gefs-pds/gefs.YYYYMMDD/HH/wave/gridded/gefs.wave.tHHz.p##.global.0p25.f###.grib2.idx


In [39]:
files = fs.glob("noaa-gefs-pds/gefs.20240101/00/chem/*/*.idx")

df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        # .str.split("/")
        # .list.last()
        .str.replace(r"gefs.\d{8}/\d\d", "gefs.YYYYMMDD/HH")
        .str.replace(r"f\d\d\d", "f###")
        .str.replace(r"t\d\dz", "tHHz")
        .alias("filename"),
    )
)
for i, f in enumerate(df["filename"].unique().sort(), start=1):
    print(f"{i:3}: {f}")

  1: noaa-gefs-pds/gefs.YYYYMMDD/HH/chem/pgrb2ap25/gefs.chem.tHHz.a2d_0p25.f###.grib2.idx
  2: noaa-gefs-pds/gefs.YYYYMMDD/HH/chem/pgrb2ap5/gefs.chem.tHHz.a3d_0p50.f###.grib2.idx


In [26]:
files = fs.glob("noaa-gefs-pds/gefs.20240101/00/atmos/*/*idx")

df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        # .str.split("/")
        # .list.last()
        .str.replace(r"gefs.\d{8}/\d\d", "gefs.YYYYMMDD/HH")
        .str.replace(r"f\d\d\d", "f###")
        .str.replace(r"gep\d\d", "gepXX")
        .str.replace(r"t\d\dz", "tHHz")
        .alias("filename"),
    )
)
for i, f in enumerate(df["filename"].unique().sort(), start=1):
    print(f"{i:3}: {f}")

  1: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2ap5/geavg.tHHz.pgrb2a.0p50.f###.idx
  2: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2ap5/gec00.tHHz.pgrb2a.0p50.f###.idx
  3: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2ap5/gepXX.tHHz.pgrb2a.0p50.f###.idx
  4: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2ap5/gespr.tHHz.pgrb2a.0p50.f###.idx
  5: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2bp5/gec00.tHHz.pgrb2b.0p50.f###.idx
  6: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2bp5/gepXX.tHHz.pgrb2b.0p50.f###.idx
  7: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2sp25/geavg.tHHz.pgrb2s.0p25.f###.idx
  8: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2sp25/gec00.tHHz.pgrb2s.0p25.f###.idx
  9: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2sp25/gepXX.tHHz.pgrb2s.0p25.f###.idx
 10: noaa-gefs-pds/gefs.YYYYMMDD/HH/atmos/pgrb2sp25/gespr.tHHz.pgrb2s.0p25.f###.idx


In [54]:
fs.glob("noaa-gefs-pds/gefs.20200922/00/*/*.idx")

['noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2aanl.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af00.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af06.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af102.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af108.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af114.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af12.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af120.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af126.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af132.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af138.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af144.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af150.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.pgrb2af156.idx',
 'noaa-gefs-pds/gefs.20200922/00/pgrb2a/gec00.t00z.p

In [13]:
files = fs.glob("noaa-gefs-pds/gefs.20200922/00/*/*")

df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        # .str.split("/")
        # .list.last()
        .str.replace(r"gefs.\d{8}/\d\d", "gefs.YYYYMMDD/HH")
        .str.replace(r"f\d\d\d", "f##")
        .str.replace(r"f\d\d", "f##")
        .str.replace(r"gep\d\d", "gepXX")
        .str.replace(r"t\d\dz", "tHHz")
        .alias("filename"),
    )
)
for i, f in enumerate(df["filename"].unique().sort(), start=1):
    print(f"{i:3}: {f}")

  1: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2a/gec00.tHHz.pgrb2aanl.idx
  2: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2a/gec00.tHHz.pgrb2af##.idx
  3: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2a/gepXX.tHHz.pgrb2aanl.idx
  4: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2a/gepXX.tHHz.pgrb2af##.idx
  5: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2b/gec00.tHHz.pgrb2banl.idx
  6: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2b/gec00.tHHz.pgrb2bf##.idx
  7: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2b/gepXX.tHHz.pgrb2banl.idx
  8: noaa-gefs-pds/gefs.YYYYMMDD/HH/pgrb2b/gepXX.tHHz.pgrb2bf##.idx


In [3]:
files = fs.glob("noaa-gfs-bdp-pds/gfs.20240401/*/atmos/*.idx")
len(files)

5356

In [40]:
files = fs.glob("noaa-gfs-bdp-pds/gfs.20240401/*/wave/*/*.idx")
len(files)

5852

In [43]:
files = fs.glob("noaa-gfs-bdp-pds/gdas.20240401/*/atmos/*.idx")
len(files)

100

In [44]:
df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        .str.split("/")
        .list.last()
        .str.replace(r"f\d\d\d", "f###")
        .str.replace(r"t\d\dz", "tHHz")
        .alias("filename"),
    )
)
for i, f in enumerate(df["filename"].unique().sort(), start=1):
    print(f"{i:3}: {f}")

  1: gdas.tHHz.pgrb2.0p25.f###.idx
  2: gdas.tHHz.pgrb2.1p00.anl.idx
  3: gdas.tHHz.pgrb2.1p00.f###.idx
  4: gdas.tHHz.sfluxgrbf###.grib2.idx


In [9]:
df = (
    pl.DataFrame({"files": files})
    .filter(
        pl.col("files").str.ends_with(".idx"),
    )
    .with_columns(
        pl.col("files").str.split("/").list.last(),
        pl.col("files")
        .str.split("/")
        .list.last()
        .str.split(".")
        .list.to_struct()
        .alias("parts"),
    )
    .unnest("parts")
)
df

files,field_0,field_1,field_2,field_3,field_4,field_5
str,str,str,str,str,str,str
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f000""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f003""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f006""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f009""","""idx"""
"""gfs.t00z.goessimpgrb2.0p25.f01…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f012""","""idx"""
…,…,…,…,…,…,…
"""gfs.t00z.sfluxgrbf372.grib2.id…","""gfs""","""t00z""","""sfluxgrbf372""","""grib2""","""idx""",null
"""gfs.t00z.sfluxgrbf375.grib2.id…","""gfs""","""t00z""","""sfluxgrbf375""","""grib2""","""idx""",null
"""gfs.t00z.sfluxgrbf378.grib2.id…","""gfs""","""t00z""","""sfluxgrbf378""","""grib2""","""idx""",null


In [39]:
df1 = df.filter(pl.col("field_4").is_null().not_())
df1

files,field_0,field_1,field_2,field_3,field_4
str,str,str,str,str,str
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f000"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f003"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f006"""
"""gfs.t00z.goessimpgrb2.0p25.f00…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f009"""
"""gfs.t00z.goessimpgrb2.0p25.f01…","""gfs""","""t00z""","""goessimpgrb2""","""0p25""","""f012"""
…,…,…,…,…,…
"""gfs.t00z.pgrb2full.0p50.f375""","""gfs""","""t00z""","""pgrb2full""","""0p50""","""f375"""
"""gfs.t00z.pgrb2full.0p50.f378""","""gfs""","""t00z""","""pgrb2full""","""0p50""","""f378"""
"""gfs.t00z.pgrb2full.0p50.f381""","""gfs""","""t00z""","""pgrb2full""","""0p50""","""f381"""


In [46]:
df1["field_0"].unique().to_list(), df1["field_1"].unique().to_list()

(['gfs'], ['t00z'])

In [47]:
df1["field_2"].unique()

field_2
str
"""goessimpgrb2"""
"""syndata"""
"""pgrb2full"""
"""pgrb2"""
"""pgrb2b"""


In [48]:
df1["field_3"].unique()

field_3
str
"""tcvitals"""
"""0p25"""
"""1p00"""
"""0p50"""


In [52]:
df1["field_4"].unique().sort()

field_4
str
"""anl"""
"""f000"""
"""f001"""
"""f002"""
"""f003"""
…
"""f375"""
"""f378"""
"""f381"""


In [53]:
df2 = df.filter(pl.col("field_4").is_null())
df2

files,field_0,field_1,field_2,field_3,field_4
str,str,str,str,str,str
"""gfs.t00z.goessimpgrb2f000.grd2…","""gfs""","""t00z""","""goessimpgrb2f000""","""grd221""",null
"""gfs.t00z.goessimpgrb2f003.grd2…","""gfs""","""t00z""","""goessimpgrb2f003""","""grd221""",null
"""gfs.t00z.goessimpgrb2f006.grd2…","""gfs""","""t00z""","""goessimpgrb2f006""","""grd221""",null
"""gfs.t00z.goessimpgrb2f009.grd2…","""gfs""","""t00z""","""goessimpgrb2f009""","""grd221""",null
"""gfs.t00z.goessimpgrb2f012.grd2…","""gfs""","""t00z""","""goessimpgrb2f012""","""grd221""",null
…,…,…,…,…,…
"""gfs.t00z.wgne.f168""","""gfs""","""t00z""","""wgne""","""f168""",null
"""gfs.t00z.wgne.f171""","""gfs""","""t00z""","""wgne""","""f171""",null
"""gfs.t00z.wgne.f174""","""gfs""","""t00z""","""wgne""","""f174""",null


In [55]:
df2["field_0"].unique().to_list(), df1["field_1"].unique().to_list()

(['gfs'], ['t00z'])

In [74]:
df2["field_2"].str.replace(r"f\d\d\d$", "").unique().sort()

field_2
str
"""goessimpgrb2"""
"""sfluxgrb"""
"""wgne"""


In [71]:
df2["field_3"].unique().sort()

field_3
str
"""f003"""
"""f006"""
"""f009"""
"""f012"""
"""f015"""
…
"""f174"""
"""f177"""
"""f180"""


In [1]:
import requests

#
files = [
    "2023/20231230/gfs.0p25.2023123000.f384.grib2",
    "2023/20231230/gfs.0p25.2023123006.f378.grib2",
]
#
# download the data file(s)
for file in files:
    idx = file.rfind("/")
    if idx > 0:
        ofile = file[idx + 1 :]
    else:
        ofile = file

    print(file, "idx", idx)

    # response = requests.get("https://data.rda.ucar.edu/d084001/" + file)
    # with open(ofile, "wb") as f:
    #    f.write(response.content)

2023/20231230/gfs.0p25.2023123000.f384.grib2 idx 13
2023/20231230/gfs.0p25.2023123006.f378.grib2 idx 13


In [3]:
file.rfind("/")
ofile

'gfs.0p25.2023123006.f378.grib2'

In [1]:
from herbie import Herbie

In [5]:
H = Herbie("2023-01-01", model="hrrr", overwrite=True)
H.SOURCES

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


{'aws': 'https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20230101/conus/hrrr.t00z.wrfsfcf00.grib2',
 'nomads': 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/hrrr/prod/hrrr.20230101/conus/hrrr.t00z.wrfsfcf00.grib2',
 'google': 'https://storage.googleapis.com/high-resolution-rapid-refresh/hrrr.20230101/conus/hrrr.t00z.wrfsfcf00.grib2',
 'azure': 'https://noaahrrr.blob.core.windows.net/hrrr/hrrr.20230101/conus/hrrr.t00z.wrfsfcf00.grib2',
 'pando': 'https://pando-rgw01.chpc.utah.edu/hrrr/sfc/20230101/hrrr.t00z.wrfsfcf00.grib2',
 'pando2': 'https://pando-rgw02.chpc.utah.edu/hrrr/sfc/20230101/hrrr.t00z.wrfsfcf00.grib2'}

In [6]:
H.grib_source

'aws'

In [7]:
H.get_remoteFileName

'hrrr.t00z.wrfsfcf00.grib2'

In [22]:
H.grib

PosixPath('/home/blaylock/data/gfs/20230101/gfs.t00z.pgrb2.0p25.f000')

In [21]:
H.source

AttributeError: 'Herbie' object has no attribute 'source'

In [20]:
H.get_remoteFileName

'gfs.0p25.2023010100.f000.grib2'

In [1]:
from herbie import Herbie

In [9]:
H = Herbie("2020-09-01", model="gefs", member=0, priority="aws")
H.SOURCES

💔 Did not find ┊ model=gefs ┊ product=atmos.5 ┊ 2020-Sep-01 00:00 UTC F00


{'aws': 'https://noaa-gefs-pds.s3.amazonaws.com/gefs.20200901/00/pgrb2a/gec00.t00z.pgrb2a.0p50.f00'}

In [ ]:
Herbie("2021-01-01", model="gefs", member=0)